In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
sns.set(style="whitegrid")
from pySankey import sankey
import plotly.graph_objects as go
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
#pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
#df = pd.read_csv('dataTP1.csv')
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

In [4]:
train.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currenc

In [5]:
test.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date', 'TRF',
       'Total_Amount_Currenc

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   object 
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

* Contamos con un total de:
> * 16947 Registros en nuestro set de datos de *train*. 
> * 2551 Registros en nuestro set de datos de *test*.
> * Los 2551 registros a predecir en test, representan un 15% en cantidad de registros que tenemos en train (16947). En cantidad de oportunidades como son los porcentajes?

* Cuántas Oportunidades distintas tengo en *train* y *test*?

In [8]:
len(set(train.Opportunity_Name))

9841

In [9]:
len(set(test.Opportunity_Name))

1567

> * Se observa que las oportunidades a predecir en el set de *test* representan un 15,9% en cantidad de oportunidades que tenemos en *train*, manteniendo casi la misma proporción que teníamos con la cantidad de registros.

 * Veamos si existen registros duplicadas y para eso comparamos todos los valores existentes cada una de las filas sin tener en cuenta la columna *ID* que es el id único del registro o fila.

In [10]:
train[train.duplicated(subset=list(train.columns)[1:],keep=False)].shape[0]

619

In [11]:
test[test.duplicated(subset=list(test.columns)[1:],keep=False)].shape[0]

187

* Tenemos 619 Registros de *train* y 187 Registros de *test* que se repiten en nuestro set de datos.

* Respecto a esto: en nuestro set de *test* no podemos eliminar duplicados ya que son a todos esos registros a los que debemos predecir el *Stage*. En cambio en *train* podría pasar que filtrando estos casos repetidos podamos llegar a tener una mejor performance en nuestras predicciones ya que estaré entrenando con registros distintos y saco duplicados que no aportarían a priori información al entrenamiento sino mas bien algo de ruido. (Probar y en el Informe comentar si hay alguna mejora filtrando duplicados)

* Si nos quedamos en *train* con solo con la primera aparición de esos Registros duplicados, cuantos Registros en total debería filtrar?

In [12]:
train[train.duplicated(subset=list(train.columns)[1:],keep='first')].shape[0]

448

* A cuántas oportunidades distintas corresponden esos registros duplicados?

In [13]:
len(set(train[train.duplicated(subset=list(train.columns)[1:],keep=False)].Opportunity_Name))

131

In [14]:
# Vamos a eliminar duplicados excepto los Registros que correspondad a la primera aparición.
#train = train.drop_duplicates(subset=list(train.columns)[1:],keep='first') 
#len(set(train.ID))

* Si realizamos el filtrado de Registros repetidos en *train*, conservamos las mismas cantidades de Oportunidades (9841) pero pasamos de 16947 a 16499 Registros, osea que eliminamos 448 Registros duplicados que corresponden a un 2,64% de los registros iniciales. (Probar performance con este filtro)

### Limpieza de Regiones y Territorios 
##### (*la limpieza aplicada en train debe ser la misma que en test*)

In [15]:
train.Region.value_counts()

Japan          4892
EMEA           4664
Americas       3945
APAC           3262
Middle East     184
Name: Region, dtype: int64

In [16]:
test.Region.value_counts()

Americas       1016
EMEA            838
APAC            482
Japan           201
Middle East      14
Name: Region, dtype: int64

In [17]:
# Para los casos en que tenga a Japon como Region y a None como Territory le asigno Japón a Territory, sinó lo dejo como está
train['Territory'] = train.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)
test['Territory'] = test.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)

In [18]:
# Dejo Japón solo como territorio y como Región le asigno APAC (Asia-Pacífico)
train.loc[train['Region']=='Japan','Region']='Asia-Pacífico'
test.loc[test['Region']=='Japan','Region']='Asia-Pacífico'

In [19]:
# Cambio el nombre de la región Americas por América
train.loc[train['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
train.loc[train['Region']=='EMEA','Region']='Europa'
train.loc[train['Region']=='Middle East','Region']='Oriente Medio'
train.loc[train['Region']=='APAC','Region']='Asia-Pacífico'

# Lo mismo con test
test.loc[test['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
test.loc[test['Region']=='EMEA','Region']='Europa'
test.loc[test['Region']=='Middle East','Region']='Oriente Medio'
test.loc[test['Region']=='APAC','Region']='Asia-Pacífico'

In [20]:
train.Region.value_counts()

Asia-Pacífico    8154
Europa           4664
América          3945
Oriente Medio     184
Name: Region, dtype: int64

In [21]:
test.Region.value_counts()

América          1016
Europa            838
Asia-Pacífico     683
Oriente Medio      14
Name: Region, dtype: int64

* Un dato interesante a destacar es que en train la región predominante es Asia-Pacífico y en test en América siendo Asia-Pacífico la que ocupa la tercera posición. Este dato no es menor ya que puede que según cada región puedo sectorizar los entrenemientos y predicciones. Vamos a probar esta segmentación de predicciones para ver si obtenemos mejor performance que entrenar todo junto.

In [22]:
## Arreglamos la asignación de los territorios a sus correspondientes Regiones
train.loc[train['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
train.loc[train['Territory']=='KSA','Territory']= 'Arabia Saudita'
train.loc[(train['Territory']=='Arabia Saudita')|(train['Territory']=='Jordan')|(train['Territory']=='UAE (Dubai)')\
       |(train['Territory']=='Egypt')|(train['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
train.loc[(train['Territory']=='South America')|(train['Territory']=='SE America')|(train['Territory']=='SW America')\
       |(train['Territory']=='Central America'),'Region']= 'América'
train.loc[train['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

## Lo mismo para test
test.loc[test['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
test.loc[test['Territory']=='KSA','Territory']= 'Arabia Saudita'
test.loc[(test['Territory']=='Arabia Saudita')|(test['Territory']=='Jordan')|(test['Territory']=='UAE (Dubai)')\
       |(test['Territory']=='Egypt')|(test['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
test.loc[(test['Territory']=='South America')|(test['Territory']=='SE America')|(test['Territory']=='SW America')\
       |(test['Territory']=='Central America'),'Region']= 'América'
test.loc[test['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

### Agregamos el Hemisferio según corresponda a cada Territorio.

In [23]:
# Voy a Clasificar los Territorios según los Hemisferios que pertenecen
mis_kw_train = train['Territory']
mis_kw_test = test['Territory']

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {"Hemisferio Norte": ["NW America","Japan", "India","Italy","Germany","United Kingdom","UAE (Dubai)","Greece"\
                                     ,"Switzerland","Austria","NE America","Belgium","Tunisia",'SE America',"France","Spain","Singapore"\
                                     ,"Philippines","SW America",'Denmark', 'Central America', 'Poland', 'Norway', 'Thailand'\
                                     ,'Portugal', 'South Korea','Netherlands','Kuwait','Arabia Saudita','Cambodia', 'China (PRC)', 'KSA', 'Ireland'\
                                     , 'Jordan','Cyprus', 'Morocco', 'Turkey', 'Sweden', 'Monaco', 'Kazakhstan','Romania', 'Hungary'\
                                     , 'Taiwan','Macedonia', 'Ukraine', 'Czech Republic', 'Belarus','Vietnam', 'Saudi Arabia'\
                                     ,'Bosnia & Herzegovina', 'Kosovo', 'Luxembourg', 'Bulgaria','UAE (Others)', 'Armenia', 'Croatia'\
                                     , 'Malaysia', 'Georgia','Estonia', 'Pakistan', 'Canada', 'Central US', 'Carribean','Malta'\
                                     , 'Albania', 'Oman', 'Burma', 'Slovenia','Egypt']\
               ,"Hemisferio Sur": ["Australia",'South America','Indonesia','South East Asia','Solomon Islands','New Zealand']\
             }
key_depuradas_train=[]
key_depuradas_test=[]
keyword_processor.add_keywords_from_dict(keyword_dict)

for w in mis_kw_train:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_train.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_train.append(w)
            
for w in mis_kw_test:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_test.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_test.append(w)     

In [24]:
len(key_depuradas_train)

16947

In [25]:
len(key_depuradas_test)

2551

In [26]:
len(train['Territory'])

16947

In [27]:
len(test['Territory'])

2551

In [28]:
train['Hemisferio'] = key_depuradas_train
test['Hemisferio'] = key_depuradas_test

In [29]:
### Analizar hemisferios
#¿Filtro los hemisferios para los cuales no tenía un territorio definido?
#df = df.loc[(df['Hemisferio']!='None'),:]

In [30]:
train[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
16942,8781,Europa,Austria,Hemisferio Norte
16943,8786,Europa,Austria,Hemisferio Norte
16944,8792,Europa,Austria,Hemisferio Norte
16945,28561,América,NE America,Hemisferio Norte
16946,28318,América,NE America,Hemisferio Norte


In [31]:
test[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
2546,16345,Oriente Medio,Arabia Saudita,Hemisferio Norte
2547,15218,América,SE America,Hemisferio Norte
2548,15224,América,SE America,Hemisferio Norte
2549,7286,América,NE America,Hemisferio Norte
2550,11263,Europa,Belgium,Hemisferio Norte


### Agregamos La estación del año en que se crean las oportunidades en funcion del Hemisferio del Terrirorio al que pertenecen.

> * (podría agregar estaciones de entrega o de ultima modificación para tener más features)

In [32]:
train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])

In [33]:
def encuentra_Estacion_N(date): 
    year = str(date.year)
    seasons = {'Primavera': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Verano': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Otoño': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Verano']:
        return 'Verano'
    if date in seasons['Otoño']:
        return 'Otoño'
    else:
        return 'Invierno'   
    
def encuentra_Estacion_S(date): 
    year = str(date.year)
    seasons = {'Otoño': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Invierno': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Primavera': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Invierno']:
        return 'Invierno'
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Otoño']:
        return 'Otoño'
    else:
        return 'Verano'     

In [34]:
train.loc[train['Hemisferio']=='Hemisferio Norte','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
train.loc[train['Hemisferio']=='Hemisferio Sur','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

test.loc[test['Hemisferio']=='Hemisferio Norte','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
test.loc[test['Hemisferio']=='Hemisferio Sur','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

In [35]:
train.Estación.value_counts(normalize=True)

Primavera    0.269047
Otoño        0.266696
Verano       0.234264
Invierno     0.229993
Name: Estación, dtype: float64

In [36]:
test.Estación.value_counts(normalize=True)

Invierno     0.676210
Primavera    0.233065
Otoño        0.052823
Verano       0.037903
Name: Estación, dtype: float64

### Precio de las oportunidades

* Vamos a crear un feature con el valor total de ventas de las oportunidades (**Total_Taxable_Amount**) en una misma moneda (**dólares estadounidenses**) para que la comparación entre valores sean coherentes. También cabe destacar que en *test* tenemos casos de **Total_Taxable_Amount = 0** pero no tenemos casos de **Total_Amount = 0**, con lo cual calcularemos el precio final de cada **oportunidad** sumando dichos valores de Total_Amount para cada una de ellas. Esto lo vamos a realizar también en *train* pero tener en cuenta que train además cuenta con registros que contienen simultaneamente **Total_Taxable_Amount y Total_Taxable_Amount igual a 0**, (casos que no se dan en test) con lo cual debo tomar la mejor decición, ya sea filtrando esos casos de train o imputando algún valor a Total_Taxable_Amount.    

In [37]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,NaT,1,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,NaT,1,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno


In [38]:
train_monto_total=train.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
train_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
train_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,0,5272800.0
1,1,48230.0
2,2,83865.6
3,3,7421881.5
4,4,13357192.5


In [39]:
test_monto_total=test.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
test_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
test_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,10689,367419.0
1,10690,757783.5
2,10691,21037.5
3,10692,2169106.5
4,10693,5752.5


In [40]:
train=pd.merge(train,train_monto_total,how='inner',on='Opportunity_ID')

In [41]:
test=pd.merge(test,test_monto_total,how='inner',on='Opportunity_ID')

In [42]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,1332347.5
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,1332347.5


In [43]:
train.ASP_Currency.value_counts()

USD    7361
JPY    4522
EUR    4503
AUD     556
GBP       5
Name: ASP_Currency, dtype: int64

In [44]:
train.loc[train['ASP_Currency']=='GBP',:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local
2299,20861,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1837080.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2300,20878,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1801800.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2301,20881,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1798290.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
2302,20884,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,6/16/2014,...,5,GBP,1837080.00,GBP,7.274250e+06,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,7274250.00
3579,17651,Europa,United Kingdom,1,1,1,1,0,Bureaucratic_Code_4,10/15/2013,...,2,GBP,776081.25,EUR,9.043541e+05,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Verano,776081.25


In [45]:
def conver_EUR_dolares(monto): 
        return monto*1.131   
    
def conver_JPY_dolares(monto): 
        return monto*0.00899 

def conver_AUD_dolares(monto): 
        return monto*0.7076 
    
def conver_GBP_dolares(monto): 
        return monto*1.3181 

def conver_USD_dolares(monto): 
        return monto*1

In [46]:
train.loc[train['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

test.loc[test['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

In [47]:
train.head(20)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares
0,27761,Europa,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,None,NaN,5272800.0,5.963537e+06
1,27760,Europa,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,None,NaN,48230.0,5.454813e+04
2,27446,América,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,83865.6,8.386560e+04
3,16808,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,7421881.5,7.421882e+06
4,16805,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,13357192.5,1.335719e+07
5,16802,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,USD,14838277.5,USD,14838277.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,14838277.5,1.483828e+07
6,16799,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,USD,2659494.6,USD,2659494.6,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,2659494.6,2.659495e+06
7,27455,América,NW America,1,1,1,1,0,Bureaucratic_Code_4,4/21/2015,...,USD,50688.0,USD,50688.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,50688.0,5.068800e+04
8,24353,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,JPY,15600.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03
9,24355,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,JPY,4400.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03


In [48]:
test.head(10)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares
0,6140,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415551
1,6146,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415551
2,6151,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415551
3,6118,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053
4,6124,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053
5,6129,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,0,EUR,126555.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053
6,6135,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,0,EUR,118156.5,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053
7,6218,Europa,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,0,EUR,112464.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053
8,11530,América,NE America,1,1,0,0,0,Bureaucratic_Code_4,2/16/2019,...,0,USD,21037.5,USD,21037.5,Prod_Category_A_None,Hemisferio Norte,Primavera,21037.5,21037.5
9,20314,América,NW America,1,1,1,0,0,Bureaucratic_Code_2,7/27/2013,...,1,USD,342490.5,USD,2169106.5,Prod_Category_A_None,Hemisferio Norte,Primavera,2169106.5,2.16911e+06


In [49]:
train.groupby('Opportunity_ID').agg({'total_en_dolares':'count'}).reset_index()

,Opportunity_ID,total_en_dolares
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9836,12799,1
9837,12800,2
9838,12801,3
9839,12802,1


### Sales_Contract_No es equivalente a Stage y no se permite utilizar este feature!!!! (dejo comentado la demostración gráfica de que son casi lo mismo)

In [188]:
#train.Sales_Contract_No.value_counts().head()

In [189]:
#train['truco']='Sin Contrato'

In [190]:
#train.loc[train['Sales_Contract_No'].isnull(),:]

In [191]:
#train.loc[train['Sales_Contract_No']!='None','truco']='Con Contrato'

In [192]:
#train.truco.value_counts()

In [193]:
#train=train.loc[(train['Stage']!='Proposal') & (train['Stage']!='Negotiation') & (train['Stage']!='Qualification'),:].copy()

In [194]:
''' train['veces'] = 1
mi_map=pd.crosstab(train['truco'], train['Stage'],values=train.veces,aggfunc=np.sum)

fig, ax = plt.subplots(figsize=(10,10))
g = sns.heatmap(mi_map, cmap="viridis", ax=ax, linewidths=0.005, linecolor='black',annot=True,fmt='g')
g.set_title("Cantidad de Oportunidades Exitosas y No Exitosas según si hubo o no Contrato", fontsize=16)
g.set_xlabel("Tipo de Stage", fontsize=16, labelpad=10)
g.set_ylabel("Región", fontsize=16, labelpad=16, rotation=90)
g.set_xticklabels(g.get_xticklabels(), fontsize=13)
g.set_yticklabels(g.get_yticklabels(), fontsize=13, rotation=0)
g.title.set_position([0.5, 1.05]);
#plt.savefig('graficos/7.png');
plt.show();'''

' train[\'veces\'] = 1\nmi_map=pd.crosstab(train[\'truco\'], train[\'Stage\'],values=train.veces,aggfunc=np.sum)\n\nfig, ax = plt.subplots(figsize=(10,10))\ng = sns.heatmap(mi_map, cmap="viridis", ax=ax, linewidths=0.005, linecolor=\'black\',annot=True,fmt=\'g\')\ng.set_title("Cantidad de Oportunidades Exitosas y No Exitosas según si hubo o no Contrato", fontsize=16)\ng.set_xlabel("Tipo de Stage", fontsize=16, labelpad=10)\ng.set_ylabel("Región", fontsize=16, labelpad=16, rotation=90)\ng.set_xticklabels(g.get_xticklabels(), fontsize=13)\ng.set_yticklabels(g.get_yticklabels(), fontsize=13, rotation=0)\ng.title.set_position([0.5, 1.05]);\n#plt.savefig(\'graficos/7.png\');\nplt.show();'

### Stage

In [50]:
train.Stage.value_counts()

Closed Won       9533
Closed Lost      7350
Proposal           35
Negotiation        18
Qualification      11
Name: Stage, dtype: int64

* En el set de datos de *train* se observa que en la variable Stage, además de los estados que indican si una oportunidad resultó exitosa o no (**estados finales**), tenemos otros 3 estados (Proposal, Negotiation y Qualification) que corresponden a **estados intermedios** que tienen las oportunidades dentro del pipeline que desarrolla la empresa.

* Como con esta variable se estará luego tratando de predecir la “Probabilidad de éxito” para cada oportunidad en el set de *test*, o sea ¿Cuál es la probabilidad de que la oportunidad se convierta en un caso Closed Won? . Que decisión debemos tomar con dichos estados? Dichos estados al no tener una resolución, debemos asignarle arbitrariamente algún estado final? Debemos filtrar estos casos?

> * Finalmente luego de analizar esos registros con estos estados intermedios y verificar que corresponen a oportunidades que no tuvieron aún un desenlase (exitoso o no exitoso) y tenindo en cuenta que corresponden a unos pocos casos, tomamos la decisión de filtrar estos registros y quedarnos solamente con los que tienen un estado final definido.

In [51]:
train=train.loc[(train['Stage']!='Proposal') & (train['Stage']!='Negotiation') & (train['Stage']!='Qualification'),:]

In [52]:
train.Stage.value_counts()

Closed Won     9533
Closed Lost    7350
Name: Stage, dtype: int64

### Cantidad de Items de cada Oportunidad

In [53]:
Op_cant_train = train.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_test = test.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_train.rename(columns = {'ID': 'Cantidad de Items Asociados'}, inplace = True)
Op_cant_test.rename(columns = {'ID': 'Cantidad de Items Asociados'}, inplace = True)

train=pd.merge(Op_cant_train,train,on='Opportunity_ID')
test=pd.merge(Op_cant_test,test,on='Opportunity_ID')

In [54]:
train[['ID','Region','Opportunity_ID','Cantidad de Items Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad de Items Asociados
8409,23352,América,5275,1
9428,23979,Europa,5963,2
12337,5522,Asia-Pacífico,7510,1
4907,27884,América,2911,24
13652,12969,América,8322,1


In [55]:
test[['ID','Region','Opportunity_ID','Cantidad de Items Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad de Items Asociados
457,10974,Europa,10971,3
541,19541,América,11006,8
2376,9417,Asia-Pacífico,12266,26
1865,5630,América,11958,1
1866,12372,América,11959,1


In [56]:
train.head(20)

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares
0,0,1,27761,Europa,None,1,1,1,1,0,...,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,None,NaN,5272800.0,5.963537e+06
1,1,1,27760,Europa,None,0,0,0,0,0,...,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,None,NaN,48230.0,5.454813e+04
2,2,1,27446,América,NW America,0,0,0,0,0,...,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,83865.6,8.386560e+04
3,3,1,16808,América,NW America,1,0,1,0,0,...,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,7421881.5,7.421882e+06
4,4,1,16805,América,NW America,1,0,1,0,0,...,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,13357192.5,1.335719e+07
5,5,1,16802,América,NW America,1,0,1,0,0,...,USD,14838277.5,USD,14838277.5,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,14838277.5,1.483828e+07
6,6,1,16799,América,NW America,1,0,1,0,0,...,USD,2659494.6,USD,2659494.6,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,2659494.6,2.659495e+06
7,7,1,27455,América,NW America,1,1,1,1,0,...,USD,50688.0,USD,50688.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Otoño,50688.0,5.068800e+04
8,8,5,24353,Asia-Pacífico,Japan,1,0,0,0,0,...,JPY,15600.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03
9,8,5,24355,Asia-Pacífico,Japan,1,0,0,0,0,...,JPY,4400.0,JPY,470400.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otoño,470400.0,4.228896e+03


### Precio total en dolares por Items de la Oportunidad

In [57]:
train['precio_por_items'] = train['total_en_dolares']/train['Cantidad de Items Asociados']

In [58]:
test['precio_por_items'] = test['total_en_dolares']/test['Cantidad de Items Asociados']

In [59]:
test.head()

,Opportunity_ID,Cantidad de Items Asociados,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Total_moneda_local,total_en_dolares,precio_por_items
0,10689,3,6140,Europa,Germany,1,1,1,1,0,...,EUR,162240.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415551,138517
1,10689,3,6146,Europa,Germany,1,1,1,1,0,...,EUR,78624.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415551,138517
2,10689,3,6151,Europa,Germany,1,1,1,1,0,...,EUR,126555.0,EUR,367419.0,Prod_Category_A_None,Hemisferio Norte,Primavera,367419.0,415551,138517
3,10690,5,6118,Europa,Germany,1,1,1,1,0,...,EUR,243360.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053,171411
4,10690,5,6124,Europa,Germany,1,1,1,1,0,...,EUR,157248.0,EUR,757783.5,Prod_Category_A_None,Hemisferio Norte,Primavera,757783.5,857053,171411


### Fechas

In [ ]:
train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])

train['Last_Activity']=pd.to_datetime(train['Last_Activity'])
test['Last_Activity']=pd.to_datetime(test['Last_Activity'])

train['Quote_Expiry_Date']=pd.to_datetime(train['Quote_Expiry_Date'])
test['Quote_Expiry_Date']=pd.to_datetime(test['Quote_Expiry_Date'])

train['Last_Modified_Date']=pd.to_datetime(train['Last_Modified_Date'])
test['Last_Modified_Date']=pd.to_datetime(test['Last_Modified_Date'])

train['Planned_Delivery_Start_Date']=pd.to_datetime(train['Planned_Delivery_Start_Date'])
test['Planned_Delivery_Start_Date']=pd.to_datetime(test['Planned_Delivery_Start_Date'])

train['Planned_Delivery_End_Date']=pd.to_datetime(train['Planned_Delivery_End_Date'])
test['Planned_Delivery_End_Date']=pd.to_datetime(test['Planned_Delivery_End_Date'])

train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])

In [61]:
train.to_csv("train_listo.csv", index=False)
test.to_csv("test_listo.csv", index=False)

In [68]:
train['pred']=0

In [69]:
train.pred.value_counts()

0    16883
Name: pred, dtype: int64

In [70]:
train.loc[train['Stage']=='Closed Won','pred']=1
train.loc[train['Stage']!='Closed Won','pred']=0

In [71]:
train.pred.value_counts()

1    9533
0    7350
Name: pred, dtype: int64

In [76]:
train['target']=1

In [77]:
log_loss(train.pred, train.target)

16.64715137451298

In [78]:
train = train.drop_duplicates(subset='Opportunity_ID',keep='first') 

In [79]:
log_loss(train.pred, train.target)

16.64715137451298